In [1]:
# import the necessary packages

# from trainerCNN_SmallVGGNet import SmallVGGNet # custom
#Instead of from keras.models import Sequential , this time we import SmallVGGNet

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD

from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
import cv2, pickle, os, random

In [2]:
data = []
labels = []

imagePaths = sorted(list(paths.list_images(r'animals/')))
random.seed(42)
random.shuffle(imagePaths)

#progressBar
pbar = tqdm(total=len(imagePaths), desc="🔃 Images: ")
for imagePath in imagePaths:
    image = cv2.resize(cv2.imread(imagePath), (64, 64)) #load and resize
    data.append(image)
    
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)
    
    pbar.update(1)
pbar.close()

🔃 Images:   0%|          | 0/3000 [00:00<?, ?it/s]

In [3]:
#scale the raw pixel intensities range 0 to 1
data = np.array(data, dtype = "float")/255.0
labels = np.array(labels)

In [4]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

In [5]:
# convert the labels from integers to vectors (for 2-class, binary
# classification you should use Keras' to_categorical function
# instead as the scikit-learn's LabelBinarizer will not return a
# vector)
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [6]:
#image generator for data augmentation 
#Data augmentation is often a critical step to:
    # -Avoiding overfitting , Ensuring your model generalizes well
    
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, 
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")

In [14]:
print(data.shape)
print(labels.shape)

(3000, 64, 64, 3)
(3000,)


In [7]:
# initialize our VGG-like Convolutional Neural Network
from trainerCNN_SmallVGGNet import SmallVGGNet
model = SmallVGGNet.build(width=64, height=64, channel=3, classes=len(lb.classes_))

In [8]:
initLR = 0.01
EPOCHS = 75
BS = 32

print("🔃: init model and optimizer")
opt = SGD(lr = initLR, decay= initLR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

🔃: init model and optimizer


### <center>  Train the network

In [9]:
H = model.fit(x = aug.flow(testX, trainY, batch_size=BS),
             validation_data = (testX, testY),
             steps_per_epoch=len(trainX)//BS,
             epochs=EPOCHS)

ValueError: `x` (images tensor) and `y` (labels) should have the same length. Found: x.shape = (750, 64, 64, 3), y.shape = (2250, 3)